## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, use_current_place_point_only, use_current_sub_binders_only, finish_L_shape, set_l_shape_object, ModeSwitcher

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8249903 , -0.02537787, -0.56457686,  0.6518986 ],
        [ 0.17256013,  0.9625954 ,  0.20888528, -0.15673575],
        [ 0.53815806, -0.2697518 ,  0.798511  ,  0.09325856],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = load_json("data/xyz_rpy_robots.json")
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
# save_json("data/xyz_rpy_robots.json", xyz_rpy_robots)

## init planning pipeline

In [7]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [8]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


## add indy tool

In [9]:
add_indy_sweep_tool(gscene, "indy0", face_name="brush_face")

## add sweep face

In [10]:
R_CURVE = 0.19
CURVE_WIDTH = 0.19
CURVE_LENGTH = 0.24
CURVE_HEIGHT = 0.026
curve_base = s_builder.detect_and_register(item_names=["curve_base"], level_mask=[DetectionLevel.ENVIRONMENT])["curve_base"]
curve_face = gscene.create_safe(GEOTYPE.CYLINDER, "curve_face",
                                link_name=curve_base.link_name,
                                dims=(R_CURVE*2,R_CURVE*2,CURVE_LENGTH), 
                                center=(0,0,curve_base.dims[2]/2+CURVE_HEIGHT-R_CURVE), rpy=(0,-np.pi/2,0),
                                color=curve_base.color, display=True, fixed=True, collision=True, parent="curve_base")

In [11]:
gscene.update_markers_all()

## add wp

In [12]:
TOOL_DIM = [0.06, 0.13]
TACK_WIDTH = 0.17
TRC_THIC = 0.01
CURVE_ANGLE = np.pi/10
TRACK_NUM = np.ceil(np.divide(CURVE_LENGTH-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (CURVE_LENGTH-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = np.multiply((cos(CURVE_ANGLE), sin(CURVE_ANGLE), 0), R_CURVE-TRC_THIC/2) - (0,0,CURVE_LENGTH/2-TOOL_DIM[1]/2)
WP_REF_B = np.array([WP_REF_A[0], -WP_REF_A[1], WP_REF_A[2]])

In [13]:

track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[1]/2,TOOL_DIM[0]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,0,TRACK_STEP*i_trc]),rpy=(0,np.pi/2,CURVE_ANGLE), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="curve_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[1]/2,TOOL_DIM[0]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,0,TRACK_STEP*i_trc]),rpy=(0,np.pi/2,-CURVE_ANGLE), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="curve_face")
    vertical = gscene.create_safe(GEOTYPE.PLANE, "vertical{}".format(i_trc+1), "base_link", 
                              (R_CURVE*3,R_CURVE*3,1e-6), 
                             center=(R_CURVE-CURVE_HEIGHT,0,WP_REF_A[2]+TRACK_STEP*i_trc),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=False, fixed=True, collision=False, parent="curve_face")
    face = gscene.create_safe(GEOTYPE.CYLINDER, "face{}".format(i_trc+1), "base_link", 
                              ((R_CURVE+TRC_THIC)*2,(R_CURVE+TRC_THIC)*2,TACK_WIDTH), 
                             center=(0,0,WP_REF_A[2]+TRACK_STEP*i_trc),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="curve_face")
    track_list.append((wp1, wp2, vertical, face))

In [14]:
gscene.update_markers_all()

In [15]:
track_block = gscene.create_safe(GEOTYPE.BOX, "track_block", "base_link", (curve_base.dims[0], curve_base.dims[1]*2, curve_base.dims[1]*2), 
                             (0,0,-curve_base.dims[1]),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=True, parent="curve_base")

## add box

In [16]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [17]:
gscene.update_markers_all()

## Register binders

In [18]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [19]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="curve_face", gname="curve_face", _type=PlacePlane)

## add objects

In [20]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [21]:
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [22]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [23]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, vertical, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="curve_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,TRC_THIC/2], [0,0,-np.pi/2]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,TRC_THIC/2], [0,0,-np.pi/2])}, 
                                   geometry_vertical=vertical, clearance=[face])
    sweep_list.append(sweep_)

### planners

In [24]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [25]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [26]:
mplan.motion_filters = checkers_all

In [27]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [28]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

(0, 0)


In [29]:
pscene.subject_name_list

['sweep1', 'sweep2']

# Node Sampler

In [30]:
# from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

# tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
# tplan.parent_node_sampler = UniformNodeSampler(3)
# # tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [31]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
# tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan & show

In [32]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()

In [33]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        with gtimer.block(gname):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)
            
initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=False)
gscene.update_markers_all()

# remove place points and sub-binders except for the current ones
use_current_place_point_only(pscene, initial_state)
use_current_sub_binders_only(pscene, initial_state)
tplan.prepare()
mplan.update_gscene()

In [34]:
print(gtimer)

detect: 	200.6 ms/1 = 200.6 ms (200.646/200.646)



In [35]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(TRACK_NUM):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("curve_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["curve_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=True, timeout=5, timeout_constrained=10
                 )
    
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

(0, 0)
Use 36/36 agents
try: 0 - (0, 0)->(1, 0)
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
=============== try reaching goal from (1, 0) =================
result: 0 - (0, 0)->(1, 0) = success
branching: 0->1 (0.2/50.0 s, steps/err: 26(102.843999863 ms)/0.00160058501431)
try: 1 - (1, 0)->(2, 0)
try: 1 - (1, 0)->(2, 0)
transition motion tried: True
try transition motion
=============== try reaching goal from (1, 0) =================
result: 0 - (0, 0)->(1, 0) = success
try constrained motion
try: 2 - (1, 0)->(2, 0)
try constrained motion
transition motion tried: True
branching: 0->2 (0.27/50.0 s, steps/err: 26(109.944820404 ms)/0.00134029237305)
try: 0 - (0, 0)->(0, 1)
=============== try reaching goal from (1, 0) =================
try: 3 - (1, 0)->(2, 0)
try constrained motion
result: 0 - (0, 0)->(1, 0) = success
branching: 0->3 (0.32/50.0 s, steps/err: 26(115.867

try: 1 - (1, 0)->(2, 0)
constrained motion tried: True
try joint motion
try: 2 - (1, 0)->(2, 0)
try transition motion
constrained motion tried: True
joint motion tried: True
constrained motion tried: True
======================= terminated 4: first answer acquired ===============================
result: 7 - (1, 0)->(2, 0) = success
result: 1 - (1, 0)->(2, 0) = success
result: 3 - (1, 0)->(2, 0) = success
try constrained motion
try constrained motion
constrained motion tried: True
branching: 1->25 (1.06/50.0 s, steps/err: 22(541.615009308 ms)/0.00111486369593)
branching: 7->24 (1.06/50.0 s, steps/err: 18(498.336791992 ms)/0.00156928525625)
branching: 3->22 (1.06/50.0 s, steps/err: 35(411.904811859 ms)/0.00127304868922)
======================= terminated 24: first answer acquired from other agent ===============================
++ adding return motion to acquired answer ++
++ adding return motion to acquired answer ++
result: 6 - (0, 1)->(0, 2) = success
branching: 6->26 (1.07/50.0 s, st

======================= terminated 30: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired ===============================
======================= terminated 31: first answer acquired from other agent ===============================
joint motion tried: True
constrained motion tried: True
======================= terminated 1: first answer acquired ===============================
======================= terminated 32: first answer acquired from other agent ===============================
constrained motion tried: True
result: 3 - (1, 0)->(2, 0) = success
branching: 3->40 (1.42/50.0 s, steps/err: 40(623.65603447 ms)/0.00174608602177)
result: 1 - (1, 0)->(2, 0) = success
++ adding return motion to acquired answer ++
branching: 1->41 (1.43/50.0 s, steps/err: 43(402.322053909 ms)/0.00187040809088)
======================= terminated 8: first answer acquired ===============================
======================= t

try joint motion
try: 0 - (2, 0)->(2, 1)
try transition motion
try: 0 - (2, 0)->(2, 1)
try transition motion
transition motion tried: True
joint motion tried: True
=============== try reaching goal from (2, 1) =================
result: 0 - (2, 0)->(2, 1) = success
branching: 0->1 (0.21/50.0 s, steps/err: 32(110.243082047 ms)/0.00162278839894)
try: 1 - (2, 1)->(2, 2)
try: 1 - (2, 1)->(2, 2)
try transition motion
try constrained motion
======================= terminated 20: first answer acquired ===============================
try: 0 - (2, 0)->(2, 1)
try constrained motion
transition motion tried: True
transition motion tried: True
=============== try reaching goal from (2, 1) =================
result: 0 - (2, 0)->(2, 1) = success
=============== try reaching goal from (2, 1) =================
branching: 0->2 (0.29/50.0 s, steps/err: 27(121.100902557 ms)/0.00126511283453)
result: 0 - (2, 0)->(2, 1) = success
branching: 0->3 (0.29/50.0 s, steps/err: 65(163.733959198 ms)/0.00149745578684)


try constrained motion
++ adding return motion to acquired answer ++
try: 0 - (2, 0)->(2, 1)
try joint motion
try: 0 - (2, 0)->(2, 1)
try transition motion
transition motion tried: True
constrained motion tried: True
=============== try reaching goal from (2, 1) =================
result: 0 - (2, 0)->(2, 1) = success
branching: 0->17 (1.25/50.0 s, steps/err: 27(139.147996902 ms)/0.00209856423742)
result: 8 - (2, 1)->(2, 2) = success
try transition motion
try: 17 - (2, 1)->(2, 2)
joint motion tried: True
constrained motion tried: True
branching: 8->18 (1.28/50.0 s, steps/err: 34(634.32598114 ms)/0.00134848825751)
try: 12 - (2, 1)->(2, 2)
result: 7 - (2, 1)->(2, 2) = success
++ adding return motion to acquired answer ++
joint motion tried: True
try joint motion
branching: 7->19 (1.29/50.0 s, steps/err: 34(677.664995193 ms)/0.00129854446362)
try constrained motion
try constrained motion
try: 0 - (2, 0)->(2, 1)
constrained motion tried: True
result: 5 - (2, 1)->(2, 2) = success
++ adding re

try joint motion
constrained motion tried: True
constrained motion tried: True
result: 1 - (2, 1)->(2, 2) = success
joint motion tried: True
result: 4 - (2, 1)->(2, 2) = success
joint motion tried: True
branching: 1->34 (1.8/50.0 s, steps/err: 74(1043.52593422 ms)/0.00210859229928)
======================= terminated 35: first answer acquired from other agent ===============================
branching: 4->35 (1.8/50.0 s, steps/err: 29(1249.42994118 ms)/0.00149048280932)
joint motion tried: True
joint motion tried: True
++ adding return motion to acquired answer ++
joint motion tried: True
======================= terminated 1: first answer acquired ===============================
++ adding return motion to acquired answer ++
try joint motion
try joint motion
constrained motion tried: True
result: 6 - (2, 1)->(2, 2) = success
branching: 6->41 (1.84/50.0 s, steps/err: 39(1188.57097626 ms)/0.00190454088265)
++ adding return motion to acquired answer ++
try joint motion
joint motion tried: Tr

======================= terminated 9: first answer acquired from other agent ===============================
transition motion tried: False
result: 0 - (2, 0)->(2, 1) = fail
======================= terminated 5: first answer acquired from other agent ===============================
transition motion tried: False
result: 0 - (2, 0)->(2, 1) = fail
======================= terminated 26: first answer acquired from other agent ===============================
transition motion tried: False
result: 0 - (2, 0)->(2, 1) = fail
======================= terminated 29: first answer acquired from other agent ===============================
constrained motion tried: False
result: 1 - (2, 1)->(2, 2) = fail
======================= terminated 4: first answer acquired from other agent ===============================
constrained motion tried: False
result: 1 - (2, 1)->(2, 2) = fail
======================= terminated 14: first answer acquired from other agent ===============================
constrained moti

In [36]:
print(gtimer)

detect: 	169.9 ms/1 = 169.9 ms (169.938/169.938)
firstmove: 	1762.4 ms/1 = 1762.4 ms (1762.417/1762.417)
plan0: 	1513.6 ms/1 = 1513.6 ms (1513.592/1513.592)
initialize_memory: 	35.3 ms/2 = 17.7 ms (12.387/22.958)
init_search: 	4.3 ms/2 = 2.2 ms (1.369/2.959)
start_process: 	3046.3 ms/2 = 1523.1 ms (1490.501/1555.759)
plan1: 	1590.3 ms/1 = 1590.3 ms (1590.324/1590.324)



## mix full schedule

In [37]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

mix_schedule_safe: 	4.7 ms/1 = 4.7 ms (4.71/4.71)



### play schedule

In [38]:
ppline.play_schedule(safe_mixed, period=0.001)

(0, 0)->(0, 0)
(0, 0)->(1, 0)
(1, 0)->(2, 0)
(2, 0)->(2, 0)
(2, 0)->(2, 1)
(2, 1)->(2, 2)
(2, 2)->(2, 2)


In [38]:
# for i_ss, snode_schedule in enumerate(snode_schedule_all):
#     for i_s, snode in enumerate(snode_schedule):
#         if snode.traj is not None:
#             save_json("data/curv_{}_{}.json".format(i_ss, i_s), snode.traj[:,:6])

In [ ]:
# from pkg.utils.code_scraps import play_schedule_clearance_highlight

In [ ]:
# ppline.play_schedule(safe_mixed[:1], period=0.001)

In [ ]:
# color_off = (0.8,0.2,0.2,0.2)
# for obj_name in ppline.pscene.subject_name_list:
#     if isinstance(ppline.pscene.subject_dict[obj_name], AbstractTask):
#         actor, obj = ppline.pscene.actor_dict['brush_face'], ppline.pscene.subject_dict[obj_name]
#         for gtem in obj.clearance:
#             gtem.color = color_off
#             ppline.pscene.gscene.update_marker(gtem)
#             ppline.pscene.set_object_state(snode_schedule_all[0][0].state)
# ppline.pscene.gscene.update_markers_all()

In [ ]:
# play_schedule_clearance_highlight(ppline, safe_mixed, tcheck=tcheck, period=0.001)

### Run fully mixed schedule

In [39]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)


In [39]:
crob.grasp(True, True)
time.sleep(2)
crob.grasp(False, False)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [40]:
# crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [41]:
ppline.execute_schedule(safe_mixed, mode_switcher=ModeSwitcher(pscene))

binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
http://192.168.0.63:9990/param_setting?control_force0=1
http://192.168.0.63:9990/param_setting?control_force0=0
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.63)
http://192.168.0.63:9990/param_setting?control_force0=1
http://192.168.0.63:9990/param_setting?control_force0=0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.63)
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


## Plan & execute

In [ ]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(2)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("curve_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["curve_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=200, multiprocess=True, timeout=10)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        gtimer.toc("firstmove")
    t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), kwargs=dict(auto_stop=False, mode_switcher=ModeSwitcher(pscene)))
    t_exe.start()
    snode_schedule_all.append(snode_schedule)
t_exe.join()
crob.stop_tracking()

In [ ]:
print(gtimer)

In [ ]:
for _ in range(3):
    indy = crob.robot_dict['indy0']
    with indy:
        indy.wait_di(16)

    mplan.reset_log(False)
    gtimer.reset()

    with gtimer.block("detect"):
        for sname in pscene.subject_name_list:
            if isinstance(pscene.subject_dict[sname], AbstractObject):
                pscene.remove_subject(sname)
            
        gscene.clear_non_fixed()

        ## detect again
        gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
    
        ## create boxes
        box_list =  [] 
        for gname in sorted(gtem_dict.keys()):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)

        initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
        gscene.update_markers_all()

        # remove place points and sub-binders except for the current ones
        use_current_place_point_only(pscene, initial_state)
        use_current_sub_binders_only(pscene, initial_state)
        tplan.prepare()
        mplan.update_gscene()

    print(initial_state.node)

    gtimer.tic("firstmove")
    obj_num = len(gtem_dict)
    sweep_num = len(sweep_list)
    from_state = initial_state
    t_exe = None
    snode_schedule_all = []
    for sweep_idx in range(sweep_num):
        gcheck.put_banned = [track_list[sweep_idx][2]]
        sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
    #     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
        goal_nodes = [("curve_face",)*obj_num+sweep_goal]
        if sweep_idx < sweep_num-1:
            for i_s in range(obj_num):
                obj_goal = ["curve_face"]*obj_num
                obj_goal[i_s] = "grip1"
                goal_nodes += [tuple(obj_goal)+sweep_goal]
        gtimer.tic("plan{}".format(sweep_idx))
        ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                      timeout_loop=20, multiprocess=True, timeout=5)
        gtimer.toc("plan{}".format(sweep_idx))
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.tplan.sort_schedule(schedules)
        snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
        snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
        snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
        snode_schedule = mix_schedule(mplan, snode_schedule_safe)
        from_state = snode_schedule[-1].state
        if t_exe:
            t_exe.join()
        else:
            gtimer.toc("firstmove")
        t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), kwargs=dict(auto_stop=False, mode_switcher=ModeSwitcher(pscene)))
        t_exe.start()
        snode_schedule_all.append(snode_schedule)
    t_exe.join()
    crob.stop_tracking()

In [ ]:
print(gtimer)